# Stationdata

## Data preparation for heat map

First, we start by preparing our data. A dataset with GPS coordinates for each station can be found on the website of BlueBikes (https://www.bluebikes.com/system-data). After importing the dataset, we add a new column which contains the coordinates.

In [1]:
### locations of stations ###

import pandas as pd
import numpy as np

# Import dataset given on website
# skip first row, contains update information
# no null entries that need to be deleted, see dataset.info()
stationdata = pd.read_csv("current_bluebikes_stations.csv",skiprows = 1, sep=",")

# add column "Coordinates" #

stationdata['Coordinates'] = list(zip(stationdata['Latitude'].round(4),stationdata['Longitude'].round(4)))

# rename column Name to start_station_name, end_station_name, so that we can use merge method next (you can only merge columns with the same name)
stationdata["end_station_name"] = stationdata["Name"]
stationdata["start_station_name"] = stationdata["Name"]
stationdata.head()

,Number,Name,Latitude,Longitude,District,Public,Total docks,Coordinates,end_station_name,start_station_name
0,W32006,160 Arsenal,42.364664,-71.175694,Watertown,Yes,11,"(42.3647, -71.1757)",160 Arsenal,160 Arsenal
1,A32019,175 N Harvard St,42.363796,-71.129164,Boston,Yes,18,"(42.3638, -71.1292)",175 N Harvard St,175 N Harvard St
2,S32035,191 Beacon St,42.380323,-71.108786,Somerville,Yes,19,"(42.3803, -71.1088)",191 Beacon St,191 Beacon St
3,C32094,2 Hummingbird Lane at Olmsted Green,42.288870,-71.095003,Boston,Yes,17,"(42.2889, -71.095)",2 Hummingbird Lane at Olmsted Green,2 Hummingbird Lane at Olmsted Green
4,S32023,30 Dane St,42.381001,-71.104025,Somerville,Yes,15,"(42.381, -71.104)",30 Dane St,30 Dane St


We also need to import the dataset containing the station and user data.

In [2]:
# import boston csv

data = pd.read_csv("boston_2019.csv", sep=",")

# convert columns start_time and end_time to datetime object
from datetime import date, time, datetime, timedelta 

data["start_time"] = pd.to_datetime(data["start_time"], infer_datetime_format=True)
data["end_time"] = pd.to_datetime(data["end_time"], infer_datetime_format=True)


### add column "duration" ###

# end_time - start_time
data['time_delta'] = (data.end_time - data.start_time)

# create a colume with timedelta as total hours, as a float type
data['duration_m'] = (data.end_time - data.start_time) / pd.Timedelta(minutes=1)

# create a colume with timedelta as total minutes, as a float type
data['duration_h'] = (data.end_time - data.start_time) / pd.Timedelta(hours=1)


data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522771 entries, 0 to 2522770
Data columns (total 11 columns):
 #   Column              Dtype          
---  ------              -----          
 0   start_time          datetime64[ns] 
 1   end_time            datetime64[ns] 
 2   start_station_id    int64          
 3   end_station_id      int64          
 4   start_station_name  object         
 5   end_station_name    object         
 6   bike_id             int64          
 7   user_type           object         
 8   time_delta          timedelta64[ns]
 9   duration_m          float64        
 10  duration_h          float64        
dtypes: datetime64[ns](2), float64(2), int64(3), object(3), timedelta64[ns](1)
memory usage: 211.7+ MB


,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,time_delta,duration_m,duration_h
0,2019-01-01 00:09:13.798,2019-01-01 00:15:25.336,80,179,MIT Stata Center at Vassar St / Main St,MIT Vassar St,3689,Subscriber,0 days 00:06:11.538000,6.192300,0.103205
1,2019-01-01 00:33:56.182,2019-01-01 00:38:20.880,117,189,Binney St / Sixth St,Kendall T,4142,Subscriber,0 days 00:04:24.698000,4.411633,0.073527
2,2019-01-01 00:41:54.600,2019-01-01 00:49:33.273,68,96,Central Square at Mass Ave / Essex St,Cambridge Main Library at Broadway / Trowbridg...,1628,Subscriber,0 days 00:07:38.673000,7.644550,0.127409
3,2019-01-01 00:43:32.571,2019-01-01 00:49:37.426,89,334,Harvard Law School at Mass Ave / Jarvis St,Mass Ave at Hadley/Walden,2969,Subscriber,0 days 00:06:04.855000,6.080917,0.101349
4,2019-01-01 00:49:56.464,2019-01-01 01:01:17.701,73,367,Harvard Square at Brattle St / Eliot St,Vassal Lane at Tobin/VLUS,3469,Subscriber,0 days 00:11:21.237000,11.353950,0.189232


In the following cells, we want to merge the datasets 'data' and 'stationdata'. Our goal is to create a table which contains the station names as well as the associated coordinates. We will create a table each for the start data and the end data. We will drop all unnecessary columns to get a compact dataset in the end.

In [3]:
startdata = data.merge(stationdata, how='left', on="start_station_name")

startdata["start_coo"] = startdata["Coordinates"]
startdata = startdata.drop(columns=['end_station_name_x', 'end_station_name_y', 'time_delta', 'duration_h', 'Number', 'Name', 'Latitude', 'Longitude', 'Public', 'Total docks', 'Coordinates'])

startdata.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,bike_id,user_type,duration_m,District,start_coo
0,2019-01-01 00:09:13.798,2019-01-01 00:15:25.336,80,179,MIT Stata Center at Vassar St / Main St,3689,Subscriber,6.192300,Cambridge,"(42.3621, -71.0912)"
1,2019-01-01 00:33:56.182,2019-01-01 00:38:20.880,117,189,Binney St / Sixth St,4142,Subscriber,4.411633,Cambridge,"(42.3661, -71.0863)"
2,2019-01-01 00:41:54.600,2019-01-01 00:49:33.273,68,96,Central Square at Mass Ave / Essex St,1628,Subscriber,7.644550,Cambridge,"(42.3651, -71.1031)"
3,2019-01-01 00:43:32.571,2019-01-01 00:49:37.426,89,334,Harvard Law School at Mass Ave / Jarvis St,2969,Subscriber,6.080917,Cambridge,"(42.379, -71.1199)"
4,2019-01-01 00:49:56.464,2019-01-01 01:01:17.701,73,367,Harvard Square at Brattle St / Eliot St,3469,Subscriber,11.353950,Cambridge,"(42.3732, -71.1209)"


In [4]:
enddata = data.merge(stationdata, how='left', on="end_station_name")
enddata["end_coo"] = enddata["Coordinates"]
enddata = enddata.drop(columns=['start_station_name_x', 'start_station_name_y','time_delta', 'duration_h', 'Number', 'Name', 'Latitude', 'Longitude', 'Public', 'Total docks', 'Coordinates'])
enddata.head()

,start_time,end_time,start_station_id,end_station_id,end_station_name,bike_id,user_type,duration_m,District,end_coo
0,2019-01-01 00:09:13.798,2019-01-01 00:15:25.336,80,179,MIT Vassar St,3689,Subscriber,6.192300,Cambridge,"(42.3556, -71.1039)"
1,2019-01-01 00:33:56.182,2019-01-01 00:38:20.880,117,189,Kendall T,4142,Subscriber,4.411633,Cambridge,"(42.3624, -71.085)"
2,2019-01-01 00:41:54.600,2019-01-01 00:49:33.273,68,96,Cambridge Main Library at Broadway / Trowbridg...,1628,Subscriber,7.644550,Cambridge,"(42.3734, -71.1111)"
3,2019-01-01 00:43:32.571,2019-01-01 00:49:37.426,89,334,Mass Ave at Hadley/Walden,2969,Subscriber,6.080917,Cambridge,"(42.3912, -71.1226)"
4,2019-01-01 00:49:56.464,2019-01-01 01:01:17.701,73,367,Vassal Lane at Tobin/VLUS,3469,Subscriber,11.353950,Cambridge,"(42.3854, -71.1378)"


In [5]:
print('Number of start stations: ')
print(startdata['start_station_name'].nunique())
print('Number of end stations: ')
print(enddata['end_station_name'].nunique())

Number of start stations: 
363
Number of end stations: 
364


In [6]:
startdata.to_csv('start_station_data_2019.csv')
enddata.to_csv('end_station_data_2019.csv')